In [ ]:
!date

In [ ]:
pwd

In [ ]:
import os
import numpy as np
from tqdm import tqdm

In [ ]:
def get_latent_corr(fn):
    from scipy.spatial.distance import pdist,squareform
    data = np.load(fn)
    # cmat_z = np.corrcoef(data['z_patients'])
    # cmat_s = np.corrcoef(data['z_patients'])
    cmat_z = squareform(pdist(data['z_patients']))
    cmat_s = squareform(pdist(data['s_patients']))
    triu = np.triu_indices(cmat_z.shape[0],1)
    return np.corrcoef(cmat_z[triu],cmat_s[triu])[0,1]

def get_random_string():
    import string
    import random
    return ''.join([random.choice(string.ascii_letters) for _ in range(10)])

def safe_mkdir(dirpath):
    if not os.path.exists(dirpath):
        os.mkdir(dirpath)

In [ ]:
analysis_name = 'VAE_1type_16D' + get_random_string()
#analysis_name = 'VAE_test'
#print(analysis_name)

In [ ]:
safe_mkdir(f'../scratch/{analysis_name}')

In [ ]:
save_dir_stage1 = f'../scratch/{analysis_name}/stage_1'
save_dir_stage2 = f'../scratch/{analysis_name}/stage_2'
save_dir_stage3 = f'../scratch/{analysis_name}/stage_3'

data_fn = '../Data/synth-data-01.npy'
df_fn = '../Data/synth-data-01.csv'

#data_fn = '../Data/synth-data-02.npy'
#df_fn = '../Data/synth-data-02.csv'

In [ ]:
%%time
for iter in tqdm(range(50)):
    os.system(f'python initialize_VAE-16D.py {save_dir_stage1} {data_fn} {df_fn} {iter} 10 None')

In [ ]:
files_varexp = [file for file in os.listdir(save_dir_stage1) if file.startswith('varexp_attmp_')];files_varexp.sort()
#files_latents = [file for file in os.listdir(save_dir_stage1) if file.startswith('latents_attmp_')];files_latents.sort()

varexps = np.array([np.load(os.path.join(save_dir_stage1,f))[-1] for f in files_varexp])
#vals_latents_corr = np.array([get_latent_corr(os.path.join(save_dir_stage1,f)) for f in files_latents])

file_nums = np.array([int(f.replace('varexp_attmp_','').replace('.npy','')) for f in files_varexp])
ORD = np.argsort(varexps)[-1::-1]
[print(f'{files_varexp[o]} {varexps[o]:.2f}') for o in ORD[0:20]];
IDs_best = file_nums[ORD[0:20]]
stage_2_weights = [os.path.join(save_dir_stage1,f'VAE_init_attmp_{val}') for val in IDs_best]

In [ ]:
%%time
for iter in tqdm(range(20)):
    os.system(f'python initialize_VAE-16D.py {save_dir_stage2} {data_fn} {df_fn} {iter} 100 {stage_2_weights[iter]}')